In [1]:
import librosa
import librosa.display
import IPython.display as idp
import matplotlib.pyplot as plt
import numpy as np
import csv
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

In [10]:
def mfcc_feature_vector(file_name):
  signal, sr = librosa.load(file_name)
  mfccs = librosa.feature.mfcc(y=signal, n_mfcc=13, sr=sr)
  ###We are going to try adding the deltas for more info
  delta_mfccs = librosa.feature.delta(mfccs)
  delta2_mfccs = librosa.feature.delta(mfccs, order=2)
  chroma = librosa.feature.chroma_stft(y=signal, sr=sr)
  mfccs_features = np.concatenate((mfccs, delta_mfccs, delta2_mfccs, chroma))
  mfcc_feature_vector = mfccs.flatten()
  return mfcc_feature_vector

#find the min to make consistent matrix
X_len_list = []
for i in range(800):
  num_str = '{:03d}'.format(i)
  file_name = '/content/drive/MyDrive/train/train' + num_str + '.wav'
  X_len_list.append(len(mfcc_feature_vector(file_name)))

for i in range(200):
  num_str = '{:03d}'.format(i)
  file_name = '/content/drive/MyDrive/test/test' + num_str + '.wav'
  X_len_list.append(len(mfcc_feature_vector(file_name)))

#/content/drive/MyDrive/test/test000.wav
min_len = min(X_len_list)
X = []
X_test = []

for i in range(800):
  num_str = '{:03d}'.format(i)
  file_name = '/content/drive/MyDrive/train/train' + num_str + '.wav'
  X.append(mfcc_feature_vector(file_name)[:min_len])

for i in range(200):
  num_str = '{:03d}'.format(i)
  file_name = '/content/drive/MyDrive/test/test' + num_str + '.wav'
  X_test.append(mfcc_feature_vector(file_name)[:min_len])



In [11]:
y = []



# Open the CSV file
with open('/content/drive/MyDrive/train.csv', 'r') as file:
    # Create a CSV reader object
    reader = csv.reader(file)
    # Iterate over each row in the CSV file
    for row in reader:
        # Process each row
        #print(row)  # Example: Print each row
        y.append(row[1])
    y.pop(0) #get rid of title

num_test_points = 100

X_train = X[num_test_points:]
y_train = y[num_test_points:]

X_test = X[:num_test_points]
y_test = y[:num_test_points]

In [12]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Assuming X contains the features and y contains the labels

# Training data
#X_train = ...  # 100-800th points
#y_train = ...  # Labels corresponding to 100-800th points

# Test data
#X_test = ...   # 0-100th points
#y_test = ...   # Labels corresponding to 0-100th points
for i in range(1,7):
# Initialize KNN classifier
  knn = KNeighborsClassifier(n_neighbors=i, weights='distance', algorithm='auto', leaf_size=30, p=1, metric='minkowski')

# Train the classifier
  knn.fit(X_train, y_train)

# Predict on the test data
  y_pred = knn.predict(X_test)

# Calculate accuracy
  accuracy = accuracy_score(y_test, y_pred)
  print("Accuracy:", accuracy," with ", i, "nearest neighbors")
#print("Accuracy: %", accuracy*100)

Accuracy: 0.43  with  1 nearest neighbors
Accuracy: 0.43  with  2 nearest neighbors
Accuracy: 0.43  with  3 nearest neighbors
Accuracy: 0.44  with  4 nearest neighbors
Accuracy: 0.39  with  5 nearest neighbors
Accuracy: 0.4  with  6 nearest neighbors


In [ ]:
"""
data = [['ID', 'Genre']]
for i in range(200):
  num_str = '{:03d}'.format(i)
  data.append(['test' + num_str + '.wav', knn.predict([X_test[i]])[0]])
  file_name = 'KNN_classifier.csv'

with open(file_name, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(data)
"""